# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [1]:
!nvidia-smi

Tue Jul 18 01:47:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   43C    P0    37W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [2]:
from datasets import load_dataset

data = load_dataset("daekeun-ml/naver-news-summarization-ko")

Found cached dataset csv (/opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [4]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
# data = data.map(
#     lambda x: {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
data = data.map(
    lambda x: {'text': f"### 질문: 아래의 문서를 요약해줘.\n\n###맥락: {x['document']}\n\n### 답변: {x['summary']}<|endoftext|>" }
)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-b33a50b433e46510.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-547a310d7a6e930f.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-f10a0312793924ae.arrow


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
# )
# model = AutoModelForCausalLM.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   pad_token_id=tokenizer.eos_token_id,
#   quantization_config=bnb_config, device_map={"":0}
# )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [6]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/22194 [00:00<?, ? examples/s]

Map:   0%|          | 0/2466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2740
    })
})

In [7]:
tokenizer.decode(data["train"][0]["input_ids"])

'### 질문: 아래의 문서를 요약해줘.\n\n###맥락: 앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.\n\n### 답변: 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    # target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 729403392 || trainable%: 0.21563705588032142


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [11]:
!nvidia-smi

Wed Jul 12 02:00:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   46C    P0    38W / 250W |   5707MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [11]:
# Input 길이가 2048이 넘어가는 경우 학습 불가로 인하여, 학습 전 제거. 

data = data.filter(lambda example: len(example["input_ids"]) < 2048)

for i in range(100):
    print(i, data["test"][i]["text"])

Filter:   0%|          | 0/22194 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2466 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2740 [00:00<?, ? examples/s]

0 ### 질문: 아래의 문서를 요약해줘.

###맥락: 아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.

### 답변: 아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔으며 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것으로, 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다.<|endoftext|>
1 ### 질문: 아래의 문서를 요약해줘.

###맥락: 한국형 우주발사체 누리호 가 지난 6월 21일 전남 고흥 나로우주센터 발사대에서 날아올라 성공적으로 발사됐다. 한국항공우주연구원 제공 파이낸셜뉴스 한국형 우주발사체 누리호 의 발사 성공에 힘입어 뉴스페이스 시대를 앞당기기 위한 발걸음이 빨라졌다

In [12]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

# For LLaMA
# tokenizer.pad_token_id = (
#     0  # unk. we want this to be different from the eos token
# )

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=32,
        # warmup_steps=200,
        # max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_total_limit=3,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.372900
20,2.249000
30,2.107000
40,2.034100
50,1.989500
60,1.953400
70,1.948200
80,1.931200
90,1.917500
100,1.921800


TrainOutput(global_step=516, training_loss=1.8909196299175883, metrics={'train_runtime': 18719.1001, 'train_samples_per_second': 3.529, 'train_steps_per_second': 0.028, 'total_flos': 2.4312553714640486e+17, 'train_loss': 1.8909196299175883, 'epoch': 3.0})

In [13]:
trainer.save_model("polyglot-ko_1.3b_news_3ep")

In [14]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [13]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1363: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 20, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1462: UserWarning: You 

tensor([[    0,   835, 29871,   239,   170,   139, 31406, 29901, 29871, 31346,
           238,   141,   155, 29871,   238,   133,   163, 31781, 31081, 29973,
         29871]])

In [42]:
def gen(contents):
    gened = model.generate(
        **tokenizer(
            f"""### 질문: 아래의 문서를 요약해줘.

###맥락: {contents}

### 답변:""",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        no_repeat_ngram_size=8,
        top_k=50,
        top_p=0.98,
        # temperature=0.,
    )
    print(tokenizer.decode(gened[0]))

In [46]:
gen('''삼성전자 2분기 잠정실적 발표 영업익 6천억원 예상치 웃돌아 HBM·DDR5 등 고사양 반도체 판매 호조에 DS부문 적자줄어  감산효과 본격화되는 하반기에 가격정상화·반등 본격화 예상삼성전자 서초사옥. 7일 삼성전자가 공개한 올해 2분기 잠정실적은 2008년 금융위기 이후 가장 저조했지만 시장에서는 “반도체 바닥을 확인했다”는 평가가 많았다.  업계는 올해 2분기가 시작되는 4월까지만 해도 2분기 실적이 1분기보다 대폭 악화될 것이라는 예상을 내놓았다. 특히 삼성전자가 메모리반도체 감산을 선언했음에도 반도체 가격 하락세가 이어졌고 이에 따라 수익성도 악화될 것으로 예측됐다. 1분기 실적을 뒷받침했던 갤럭시 S23 신제품 효과까지 둔화되면서 시장에서는 적자전환 가능성까지 점치는 분위기였다.  그러나 뚜껑이 열리자 “최악은 면했다”는 반응이 나왔다. 올해 2분기 삼성전자는 전분기와 비슷한 실적을 냈지만 시장 전망치를 크게 웃돌았다. 금융정보업체 에프앤가이드가 집계한 실적 컨센서스는 매출 61조8593억원, 영업이익 2818억원이었다. 영업이익이 증권사 예상치를 2배 이상 웃돈 것이다.잠정실적에 사업부별 수치는 나오지 않지만, 반도체 사업을 담당하는 디바이스솔루션 부문이 4조원대 적자를 냈을 것으로 추정된다. 메리츠증권은 삼성전자 DS부문이 4조2000억원대 적자를 기록한 것으로 분석했다. 대신 디스플레이, 모바일·네트워크, TV·생활가전 등 부문의 흑자가 DS부문의 적자폭을 메웠다는 것이 메리츠증권의 예측이다. 하만 등 기타부문에서도 3000억원 안팎의 영업이익을 예상했다.  눈에 띄는 부분은 DS부문의 적자규모가 예상을 밑돌았다는 점이다. 2분기 DS부문의 영업손실은 1분기에 비해 개선됐을 것으로 추정된다. ‘주력’으로 꼽히는 메모리반도체 부문에서는 3조원대 후반대 적자를 낸 것으로 파악된다.  생성형 인공지능 열풍으로 AI 서버 수요가 늘면서 더블데이터레이트5와 고대역폭메모리 등 고사양 제품 판매가 늘면서 적자 규모를 줄였을 것으로 추정된다. 또 적자제품 판매를 축소한 것도 영향을 미쳤다는 시각이다.  김동원 KB증권 연구원은 “2분기 D램 출하량이 전분기보다 20% 증가해 예상보다 빠르게 원가구조가 개선됐다”고 했다. 다만 시스템반도체와 파운드리 사업의 실적 부진은 이어지는 상황이다.  시장에서는 하반기에 대한 기대감도 커지고 있다. 삼성전자의 감산효과가 본격화되는 3분기부터 재고가 감소하면서 수익성이 개선될 것이라는 예상이다. 그동안 삼성전자는 쌓여있는 재고로 재고자산평가손실이 커져 수익성이 악화됐다.특히 삼성전자는 올해 3분기부터 감산규모를 최대 25% 수준까지 확대할 계획이다. 이에 따라 메모리 반도체 가격 또한 상승세를 탈 것으로 전망된다. 한동희 SK증권 연구원은 “감산 효과가 본격화되면서 출하가 이미 저점을 지나고 있다”며 “가격 반등을 모색할 수 있는 구간”이라고 내다봤다.  시장조사업체 트렌드포스는 감산효과로 올 3분기 D램 평균판매가격이 전분기보다 0~5% 하락할 것으로 예상했다. 전 분기 대비 가격 하락폭 전망치가 13~18%이었던 2분기와 비교하면 가격 하락세가 완만해질 것이라는 전망이다. 트렌드포스는 “D램 공급업체의 지속적인 감산으로 전체 D램 공급이 점차 줄고 있다”고 진단했다.  인공지능 열풍으로 HBM 등 고사양 제품 수요가 폭발적으로 늘어나고 있다는 부분도 시장 반등 기대감을 키우고 있다. 경계현 DS부문장은 최근 임직원에게 “삼성 HBM제품 시장 점유율이 여전히 50% 이상”이라며 “HBM3와 HBM3P가 내년 DS부문 이익 증가에 기여할 것”이라고 언급했다. 노근창 현대차증권 연구원은 “삼성전자는 연내 HBM3를 공급하고 기존 라인을 HBM라인으로 바꾸고 있다”고 설명했다.  최근 DS부문 적자 속에서 삼성전자는 HBM 등 분야에서의 경쟁력을 키우기 위해 인사 또한 단행한 상태다. 삼성전자는 HBM 분야를 책임지는 D램 개발실장을 교체하는 강수를 뒀다. 김동원 연구원은 “D램과 파운드리 부문 개발 총책임자 교체라는 ‘핀셋 인사’로 경쟁력 제고가 기대된다”고 분석했다.''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 아래의 문서를 요약해줘.

###맥락: 삼성전자 2분기 잠정실적 발표 영업익 6천억원 예상치 웃돌아 HBM·DDR5 등 고사양 반도체 판매 호조에 DS부문 적자줄어  감산효과 본격화되는 하반기에 가격정상화·반등 본격화 예상삼성전자 서초사옥. 7일 삼성전자가 공개한 올해 2분기 잠정실적은 2008년 금융위기 이후 가장 저조했지만 시장에서는 “반도체 바닥을 확인했다”는 평가가 많았다.  업계는 올해 2분기가 시작되는 4월까지만 해도 2분기 실적이 1분기보다 대폭 악화될 것이라는 예상을 내놓았다. 특히 삼성전자가 메모리반도체 감산을 선언했음에도 반도체 가격 하락세가 이어졌고 이에 따라 수익성도 악화될 것으로 예측됐다. 1분기 실적을 뒷받침했던 갤럭시 S23 신제품 효과까지 둔화되면서 시장에서는 적자전환 가능성까지 점치는 분위기였다.  그러나 뚜껑이 열리자 “최악은 면했다”는 반응이 나왔다. 올해 2분기 삼성전자는 전분기와 비슷한 실적을 냈지만 시장 전망치를 크게 웃돌았다. 금융정보업체 에프앤가이드가 집계한 실적 컨센서스는 매출 61조8593억원, 영업이익 2818억원이었다. 영업이익이 증권사 예상치를 2배 이상 웃돈 것이다.잠정실적에 사업부별 수치는 나오지 않지만, 반도체 사업을 담당하는 디바이스솔루션 부문이 4조원대 적자를 냈을 것으로 추정된다. 메리츠증권은 삼성전자 DS부문이 4조2000억원대 적자를 기록한 것으로 분석했다. 대신 디스플레이, 모바일·네트워크, TV·생활가전 등 부문의 흑자가 DS부문의 적자폭을 메웠다는 것이 메리츠증권의 예측이다. 하만 등 기타부문에서도 3000억원 안팎의 영업이익을 예상했다.  눈에 띄는 부분은 DS부문의 적자규모가 예상을 밑돌았다는 점이다. 2분기 DS부문의 영업손실은 1분기에 비해 개선됐을 것으로 추정된다. ‘주력’으로 꼽히는 메모리반도체 부문에서는 3조원대 후반대 적자를 낸 것으로 파악된다.  생성형 인공지능 열풍으로 AI 서버 수요가 늘면서 더블데이터레이트5와 고대역폭메모리 등 고사양 제품 판매가 늘면서 

In [40]:
import torch
torch.__version__

'2.0.1+cu117'